# 使用Mistral-7b、LangChain和ChromaDB构建RAG

## RAG四个主要步骤
1. Embedding：使用Embedding模型对文档数据进行embedding操作
2. Vector Store：将Embedding后的数据存储到向量数据库中
3. Query：将问题embedding后，通过向量数据库检索相关文档
4. Answer：使用检索到的文档和问题通过模型生成答案

# 安装相关的包
```
!pip install gradio --quiet
!pip install xformer --quiet
!pip install chromadb --quiet
!pip install langchain --quiet
!pip install accelerate --quiet
!pip install transformers --quiet
!pip install bitsandbytes --quiet
!pip install unstructured --quiet
!pip install sentence-transformers --quiet
```

## 1. 导入相关包

In [2]:
import torch
import gradio as gr

from textwrap import fill
from IPython.display import Markdown, display

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

import warnings
warnings.filterwarnings('ignore')

## 2. 加载模型

In [ ]:

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 使用4位加载模型
    bnb_4bit_compute_dtype=torch.float16,  # 使用16位计算
    bnb_4bit_quant_type="nf4",  # 使用4位量化
    bnb_4bit_use_double_quant=True,  # 使用双量化
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)  # 使用fast tokenizer
tokenizer.pad_token = tokenizer.eos_token  # 使用eos作为pad token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, 
    torch_dtype=torch.float16,  # 使用16位浮点数
    trust_remote_code=True,  # 信任远程代码
    device_map="auto",  # 自动选择设备
    quantization_config=quantization_config  # 量化配置
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)  # 从预训练模型加载生成配置
generation_config.max_new_tokens = 1024 
generation_config.temperature = 0.0001 
generation_config.top_p = 0.95 
generation_config.do_sample = True  # 采样
generation_config.repetition_penalty = 1.15  # 重复惩罚

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # 返回完整文本
    generation_config=generation_config,
)

安装xformer，以实现更高效的内存注意力实现

In [ ]:

llm = HuggingFacePipeline(pipeline=pipeline,)

### 使用Mistral-7b模型进行问答

In [ ]:

query = "Explain the difference between ChatGPT and open source LLMs in a couple of lines."
result = llm(
    query
)

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

In [ ]:

query = "What is Hiberus GenIA Ecosystem?"
result = llm(
    query
)

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

## 3. Embedding模型
Embedding模型使用阿里巴巴达摩院预训练并在Hugging Face上开源的embedding模型GTE

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

## 4. Prompt Template
PromptTemplate通过结构化Prompt格式使模型按照用户期望的格式进行输出，模板可以包括指令、few-shot例子以及适合特定任务的特定上下文和问题。

In [ ]:
template = """
[INST] <>
Act as a Machine Learning engineer who is teaching high school students.
<>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [ ]:
query = "Explain what are Deep Neural Networks in 2-3 sentences"
result = llm(prompt.format(text=query))

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

## 5. 数据加载
为了准确回答之前的问题（What is Hiberus GenIA Ecosystem?），必须将LLM与GenIA生态系统的信息联系起来。有两个网页是理解GenIA生态系统的关键。

In [ ]:
urls = [
    "https://www.hiberus.com/expertos-ia-generativa-ld",
    "https://www.hiberus.com/en/experts-generative-ai-ld"
]

loader = UnstructuredURLLoader(urls=urls)
documents = loader.load()

len(documents)
# Output

由于这两个文档数据量较大，以及超过了Mistral-7b大模型的上下文窗口大小，因此我们需要将文档按照1024个tokens大小进行切分，生成21个较小的chunks，并且为了保证上下文的连续性，chunk与chunk直接设置64个重叠tokens

In [ ]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts_chunks = text_splitter.split_documents(documents)

len(texts_chunks)

## 6. 数据注入
对数据分块之后，我们将对分块数据进行embedding并存储到向量数据库Chromdb中

In [ ]:
db = Chroma.from_documents(texts_chunks, embeddings, persist_directory="db")

数据被添加索引之后，我们可以在Prompt模板中添加RAG模型赋予营销经理专家的角色！

此外，为了将LLM与矢量数据库检索功能相结合，我们使用了关键的链接组件RetrievalQA，其中k=2。这种设置确保检索器输出两个相关的块，然后LLM在提出问题时使用这两个块来制定答案。

In [ ]:
template = """
[INST] <>
Act as an Hiberus marketing manager expert. Use the following information to answer the question at the end.
<>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",   
    retriever=db.as_retriever(search_kwargs={"k": 2}),  # 使用检索器
    return_source_documents=True,  # 返回源文档
    chain_type_kwargs={"prompt": prompt},  
)

## 7. 查询

In [ ]:
query = "What is GenAI Ecosystem?"
result_ = qa_chain(
    query
)
result = result_["result"].strip()


display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

In [ ]:
query = "Why Hiberus has created GenAI Ecosystem?"
result_ = qa_chain(
    query
)
result = result_["result"].strip()


display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

In [ ]:
# 输出源文档或参考文档
result_["source_documents"]

## 8. 问答

In [ ]:
custom_template = """You are an Hiberus Marketing Manager AI Assistant. Given the
following conversation and a follow up question, rephrase the follow up question
to be a standalone question. At the end of standalone question add this
'Answer the question in English language.' If you do not know the answer reply with 'I am sorry, I dont have enough information'.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:
"""

CUSTOM_QUESTION_PROMPT = PromptTemplate.from_template(custom_template)  # 从模板创建PromptTemplate

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)  # 创建对话缓存内存

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    memory=memory,
    condense_question_prompt=CUSTOM_QUESTION_PROMPT,
)

In [ ]:
query = "Who you are?"
result_ = qa_chain({"question": query})
result = result_["answer"].strip()

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

In [ ]:
# 查看聊天记录
memory.chat_memory.messages

## 9. 基于Gradio搭建问答UI界面

In [ ]:
def querying(query, history):
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

  qa_chain = ConversationalRetrievalChain.from_llm(
      llm=llm,
      retriever=db.as_retriever(search_kwargs={"k": 2}),
      memory=memory,
      condense_question_prompt=CUSTOM_QUESTION_PROMPT,
  )

  result = qa_chain({"question": query})
  return result["answer"].strip()

In [ ]:
iface = gr.ChatInterface(
    fn = querying,
    chatbot=gr.Chatbot(height=600),
    textbox=gr.Textbox(placeholder="What is GenAI Ecosystem?", container=False, scale=7),
    title="HiberusBot",
    theme="soft",
    examples=["Why Hiberus has created GenAI Ecosystem?",
              "What is GenAI Ecosystem?"],

    cache_examples=True,
    retry_btn="Repetir",
    undo_btn="Deshacer",
    clear_btn="Borrar",
    submit_btn="Enviar"

    )

iface.launch(share=True)